### Secondary Sort
Secondary using the 3rd key in reverse order.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%writefile test.data
4,10,3,Apple
2,2,4,Orange
6,-1,6,Lemon
0,9,18,Apple
6,-1,7,Lemon

Writing test.data


In [3]:
%%writefile test.py

'''
Input format:
....
4,10,3,Apple
2,2,4,Orange
6,-1,6,Lemon
0,9,18,Apple
6,-1,7,Lemon
....
'''
from mrjob.job import MRJob, MRStep
import mrjob
import csv
import sys

def csv_readline(line):
    return csv.reader([line]).next()

class test(MRJob):
    INTERNAL_PROTOCOL = mrjob.protocol.RawProtocol
    OUTPUT_PROTOCOL = mrjob.protocol.RawProtocol
    
    def mapper1(self, line_no, line):
        cell = csv_readline(line)
        yield cell[0], "\t".join(cell[1:])

    def reducer1(self, key, value):
        # This reducer does nothing. 
        # Just pass the mapper's output to the next step.
        for v in value:
            yield key, v

    def reducer2(self, key, value):
        yield key, ",".join(value)
    
    def steps(self):
        return [
            MRStep(mapper=self.mapper1,
                   reducer=self.reducer1),
            MRStep(reducer=self.reducer2,
                   jobconf={
                    "stream.num.map.output.key.fields":"3",
                    "mapreduce.job.output.key.comparator.class":
                        "org.apache.hadoop.mapred.lib.KeyFieldBasedComparator",
                    "mapreduce.partition.keycomparator.options":"-k1,1n -k3,3nr"
                          })]

    
if __name__ == '__main__':
    test.run()


Writing test.py


In [4]:
from test import test
mr_job = test(args=['test.data', '-r', 'hadoop'])
with mr_job.make_runner() as runner: 
    runner.run()
    print "Output:"
    for line in runner.stream_output():
        print line

Output:
0	9	18	Apple


ERROR:mrjob.fs.hadoop:STDERR: 16/02/09 22:18:11 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable




2	2	4	Orange

4	10	3	Apple

6	-1	7	Lemon,-1	6	Lemon

